In [1]:
from pyspark.sql import SparkSession

In [2]:
spark =SparkSession.builder.appName("MyGlueJob").getOrCreate()
print(spark)

25/04/17 16:56:48 WARN Utils: Your hostname, TTNPL-8203 resolves to a loopback address: 127.0.1.1; using 10.1.210.48 instead (on interface wlp0s20f3)
25/04/17 16:56:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/17 16:56:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### importing needed libs

In [3]:
from pyspark.sql.functions import col,sum, round,max,asc

In [4]:
df1 = spark.read.csv("file:///home/hdoop/notebooks/data/assessment4-mukul/Copy of covid_19_india.csv",inferSchema = True,header = True)
df2 = spark.read.csv("file:///home/hdoop/notebooks/data/assessment4-mukul/Copy of StatewiseTestingDetails.csv",inferSchema = True,header = True)

In [5]:
df1.show(n=20,truncate = False)

+---+--------+-------+--------------------+-----------------------+------------------------+-----+------+---------+
|Sno|Date    |Time   |State/UnionTerritory|ConfirmedIndianNational|ConfirmedForeignNational|Cured|Deaths|Confirmed|
+---+--------+-------+--------------------+-----------------------+------------------------+-----+------+---------+
|1  |30/01/20|6:00 PM|Kerala              |1                      |0                       |0    |0     |1        |
|2  |31/01/20|6:00 PM|Kerala              |1                      |0                       |0    |0     |1        |
|3  |01/02/20|6:00 PM|Kerala              |2                      |0                       |0    |0     |2        |
|4  |02/02/20|6:00 PM|Kerala              |3                      |0                       |0    |0     |3        |
|5  |03/02/20|6:00 PM|Kerala              |3                      |0                       |0    |0     |3        |
|6  |04/02/20|6:00 PM|Kerala              |3                      |0    

In [6]:
df2.show(n=20,truncate = False)

+----------+---------------------------+------------+--------+--------+
|Date      |State                      |TotalSamples|Negative|Positive|
+----------+---------------------------+------------+--------+--------+
|2020-04-17|Andaman and Nicobar Islands|1403.0      |1210    |12.0    |
|2020-04-24|Andaman and Nicobar Islands|2679.0      |NULL    |27.0    |
|2020-04-27|Andaman and Nicobar Islands|2848.0      |NULL    |33.0    |
|2020-05-01|Andaman and Nicobar Islands|3754.0      |NULL    |33.0    |
|2020-05-16|Andaman and Nicobar Islands|6677.0      |NULL    |33.0    |
|2020-05-19|Andaman and Nicobar Islands|6965.0      |NULL    |33.0    |
|2020-05-20|Andaman and Nicobar Islands|7082.0      |NULL    |33.0    |
|2020-05-21|Andaman and Nicobar Islands|7167.0      |NULL    |33.0    |
|2020-05-22|Andaman and Nicobar Islands|7263.0      |NULL    |33.0    |
|2020-05-23|Andaman and Nicobar Islands|7327.0      |NULL    |33.0    |
|2020-05-24|Andaman and Nicobar Islands|7327.0      |NULL    |33

#### adding the confirmedforeign percentage in df1

In [7]:
total_ConfirmedForeignNational = df1.agg(sum("ConfirmedForeignNational")).collect()[0][0]
df1_2 = df1.withColumn("ConfirmedForeignNationalPer",round(col("ConfirmedForeignNational")/total_ConfirmedForeignNational*100,2))

#### state_unionterritory with he highest positive case percentage at any given date

In [8]:
df2.show(n=20,truncate =False)

+----------+---------------------------+------------+--------+--------+
|Date      |State                      |TotalSamples|Negative|Positive|
+----------+---------------------------+------------+--------+--------+
|2020-04-17|Andaman and Nicobar Islands|1403.0      |1210    |12.0    |
|2020-04-24|Andaman and Nicobar Islands|2679.0      |NULL    |27.0    |
|2020-04-27|Andaman and Nicobar Islands|2848.0      |NULL    |33.0    |
|2020-05-01|Andaman and Nicobar Islands|3754.0      |NULL    |33.0    |
|2020-05-16|Andaman and Nicobar Islands|6677.0      |NULL    |33.0    |
|2020-05-19|Andaman and Nicobar Islands|6965.0      |NULL    |33.0    |
|2020-05-20|Andaman and Nicobar Islands|7082.0      |NULL    |33.0    |
|2020-05-21|Andaman and Nicobar Islands|7167.0      |NULL    |33.0    |
|2020-05-22|Andaman and Nicobar Islands|7263.0      |NULL    |33.0    |
|2020-05-23|Andaman and Nicobar Islands|7327.0      |NULL    |33.0    |
|2020-05-24|Andaman and Nicobar Islands|7327.0      |NULL    |33

In [9]:
df2_2 = df2.withColumn("PositiveRate",col("Positive")/col("TotalSamples")*100)

In [10]:

df2_2.where(col("Date")=='2020-04-13').where(col("PositiveRate")>10).show()

+----------+-----+------------+--------+--------+------------------+
|      Date|State|TotalSamples|Negative|Positive|      PositiveRate|
+----------+-----+------------+--------+--------+------------------+
|2020-04-13|Delhi|     15032.0|   12283|  1510.0|10.045236828100053|
+----------+-----+------------+--------+--------+------------------+



In [11]:
date_wise_max = df2_2.groupBy("Date").agg(
    max("PositiveRate").alias("PositiveRate")
).orderBy("Date")

In [16]:
df2_3 = df2_2.join(date_wise_max,["Date","PositiveRate"],"inner").select(df2_2["Date"],df2_2["State"]).orderBy("Date")

In [13]:
df1_2.where(col("ConfirmedForeignNationalPer")>0).show()

+---+--------+-------+--------------------+-----------------------+------------------------+-----+------+---------+---------------------------+
|Sno|    Date|   Time|State/UnionTerritory|ConfirmedIndianNational|ConfirmedForeignNational|Cured|Deaths|Confirmed|ConfirmedForeignNationalPer|
+---+--------+-------+--------------------+-----------------------+------------------------+-----+------+---------+---------------------------+
| 37|03/03/20|6:00 PM|           Rajasthan|                      0|                       1|    0|     0|        1|                       0.15|
| 42|04/03/20|6:00 PM|             Haryana|                      0|                       2|    0|     0|        2|                        0.3|
| 45|04/03/20|6:00 PM|           Rajasthan|                      1|                      14|    0|     0|       15|                        2.1|
| 47|05/03/20|6:00 PM|             Haryana|                      0|                       2|    0|     0|        2|                     

In [19]:
df1_2.write.format("parquet").mode("overwrite").save("file:///home/hdoop/notebooks/data/assessment4-mukul/dataset1/")

In [18]:
df2_3.write.format("parquet").mode("overwrite").save("file:///home/hdoop/notebooks/data/assessment4-mukul/dataset2/")